<a href="https://colab.research.google.com/github/Ayat-Salem/FinTech/blob/main/1636325356__assignment_w5_c1_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
pip install yfinance

In [2]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import pandas
import numpy as np
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [3]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'PFE'

In [4]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

#df = pdr.get_data_yahoo('PFE', start="2020-01-01", end="2021-01-01")
df = pdr.get_data_yahoo(stock_symbol, pandas.Timestamp('2019'), pandas.Timestamp('2021'))
df=df[df.columns[0:4]]
df = df[['Open','High','Low','Close']]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2019-01-02,40.910816,41.271347,40.455406,41.034157
2019-01-03,41.024670,41.119545,39.800758,39.886147
2019-01-04,40.265656,41.129032,40.094875,40.796963
2019-01-07,40.863377,41.385201,40.664139,41.015179
2019-01-08,41.328274,41.461102,40.939278,41.204933
...,...,...,...,...
2020-12-24,37.400002,37.570000,37.180000,37.270000
2020-12-28,37.360001,37.580002,36.680000,36.820000
2020-12-29,36.900002,37.200001,36.790001,37.049999


In [5]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'].diff()
df = df.dropna()
X = df

In [6]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)

In [7]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train= X[:index]
X_test=  X[index:]
y_train= y[:index]
y_test = y[index:]

In [8]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [9]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.11662386859975143]
1,High,[0.22751777576980714]
2,Low,[0.579554422818749]
3,Close,[-0.6749085924356258]
4,Close_15_Rolling,[-0.022326845582122872]
5,Open_1_Change,[0.05728731682775417]


In [10]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [11]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

sklearn.metrics.confusion_matrix(y_pred=predictions, y_true=y_test)

array([[25, 54],
       [12, 57]])

In [12]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

accuracy= logistic.score(X_test, y_test)
accuracy

0.5540540540540541

In [18]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
cross_val = cross_val_score(logistic, X, y, cv=5)
#cross_val_score(logistic, X_test, y_test, cv=5)
cross_val

#scores = cross_val_score(logistic, df, y, cv=5)
#print ("Cross-validated scores:", scores)


array([0.49494949, 0.29591837, 0.59183673, 0.39795918, 0.47959184])